# R workshop: <font color=blue> Generating KEGG Pathway view </font>

#### MLBI@DKU

### __0. Install required R packages (skip if they are already installed)__

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install("biocLite")
## BiocManager::install()

In [ ]:
BiocManager::install("org.Mm.eg.db")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("biomaRt")
BiocManager::install("gageData")
BiocManager::install("gage")
BiocManager::install("pathview")
install.packages("filesstrings")
install.packages("anndata")

In [43]:
## Install anndata if it was not
system('pip install anndata')

In [ ]:
devtools::install_github("combio-dku/KEGGPathviewGen4SCODA")

### __1. Load libraries and data__

In [2]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(filesstrings))
suppressPackageStartupMessages(library(pathview))
suppressPackageStartupMessages(library(gage))
suppressPackageStartupMessages(library(gageData))
suppressPackageStartupMessages(library(org.Hs.eg.db))
suppressPackageStartupMessages(library(org.Mm.eg.db))
suppressPackageStartupMessages(library(reticulate))
suppressPackageStartupMessages(library(anndata))
suppressPackageStartupMessages(library(KEGGPathviewGen4SCODA))

### __2. Load SCODA result__

In [3]:
data.dir <- "./"

flst <- list.files(data.dir)
flst

[1] "Alzheimer's disease_Normal_vs_others.png"                       
 [2] "example_human_brca_8k_results.h5ad"                             
 [3] "example_human_brca_8k_results.h5ad.tar.gz"                      
 [4] "example_kegg_pathview_gen_for_scoda.ipynb"                      
 [5] "KEGG_pathview_Epithelial cell"                                  
 [6] "optional_config_files"                                          
 [7] "scoda_practice_workshop_v2_R_kegg_pathview_code_dev-Copy1.ipynb"
 [8] "scoda_practice_workshop_v2_R_kegg_pathview_code_dev.ipynb"      
 [9] "scoda_practice_workshop_v2_R_kegg_pathview.ipynb"               
[10] "scoda_practice_workshop_v2.ipynb"                               
[11] "scoda_viz_for_py_user_v2.ipynb"                                 
[12] "scoda_workshop_example_dataset_GSE161529_33K"                   
[13] "scoda_workshop_example_dataset_GSE161529_33K_results.tar.gz"    
[14] "scoda_workshop_example_dataset_GSE161529_33K.h5ad.gz"

In [4]:
file <- 'scoda_workshop_example_dataset_GSE161529_33K_results.tar.gz'
untar(file)

In [4]:
### Load data & extract cell-gene matrix as a data.frame (rownames: cell barcode, colnames: gene symbol)
file_h5ad <- 'example_human_brca_8k_results.h5ad'

adata_t <- read_h5ad(file_h5ad)
adata_t

AnnData object with n_obs × n_vars = 12000 × 19438
    obs: 'Patient', 'Percent_mito', 'nCount_RNA', 'nFeature_RNA', 'Celltype_Major', 'Celltype_Minor', 'Celltype_Subset', 'subtype', 'gene_module', 'Calls', 'normal_cell_call', 'CNA_value', 'sample', 'condition', 'sample_rev', 'sample_ext', 'celltype_major', 'celltype_minor', 'celltype_subset', 'cnv_ref_ind', 'ploidy_score', 'ploidy_dec', 'condition_for_deg', 'sample_ext_for_deg', 'celltype_for_deg', 'celltype_for_cci', 'tumor_origin_ind'
    var: 'gene_ids', 'variable_genes', 'chr', 'spot_no'
    uns: 'CCI', 'CCI_sample', 'Celltype_marker_DB', 'DEG', 'DEG_grouping_vars', 'DEG_stat', 'GSA_down', 'GSA_up', 'GSEA', 'Pathways_DB', 'analysis_parameters', 'cnv', 'cnv_neighbors_info', 'inferploidy_summary', 'log', 'lut_sample_to_cond', 'usr_param'
    obsm: 'HiCAT_result', 'X_cnv', 'X_cnv_pca', 'X_pca', 'inferploidy_results'
    obsp: 'cnv_neighbor_graph_connectivity', 'cnv_neighbor_graph_distance'

### __3. Get mapping to KEGG pathway__

In [5]:
species <- adata_t$uns[['usr_param']][['species']]
pathways.used <- adata_t$uns[['Pathways_DB']]

df_pathways_map <- get_pathways_map( pathways.used, species, min_overlap = 0.85 )

Converting Pathways DB .. done.        


In [6]:
head(df_pathways_map)

,pw_id,pw_name,pw_id_name,pw_name_used
,<chr>,<chr>,<chr>,<chr>
hsa02010 ABC transporters,hsa02010,ABC transporters,hsa02010 ABC transporters,ABC transporters
hsa04933 AGE-RAGE signaling pathway in diabetic complications,hsa04933,AGE-RAGE signaling pathway in diabetic complications,hsa04933 AGE-RAGE signaling pathway in diabetic complications,AGE-RAGE signaling pathway in diabetic complications
hsa04152 AMPK signaling pathway,hsa04152,AMPK signaling pathway,hsa04152 AMPK signaling pathway,AMPK signaling pathway
hsa05221 Acute myeloid leukemia,hsa05221,Acute myeloid leukemia,hsa05221 Acute myeloid leukemia,Acute myeloid leukemia
hsa04520 Adherens junction,hsa04520,Adherens junction,hsa04520 Adherens junction,Adherens junction
hsa04920 Adipocytokine signaling pathway,hsa04920,Adipocytokine signaling pathway,hsa04920 Adipocytokine signaling pathway,Adipocytokine signaling pathway


### __4. Generate KEGG pathview__

In [7]:
lst.deg.all <- adata_t$uns[['DEG']]
lst.gsa.all <- adata_t$uns[['GSA_up']]

lst.fcs.all <- get_all_fold_changes( lst.deg.all, species, pval.cutoff = 1e-4 )

Getting fold changes .. 
          Aneuploid unassigned: ER+_vs_others(3211), HER2+_vs_others(2149), TNBC_vs_others(2664)
                        B cell: ER+_vs_others(55), HER2+_vs_others(15), TNBC_vs_others(172)
              Endothelial cell: ER+_vs_others(173), HER2+_vs_others(35)
                    Fibroblast: ER+_vs_others(331), HER2+_vs_others(98), TNBC_vs_others(290)
                           ILC: ER+_vs_others(94), HER2+_vs_others(37)
                    Macrophage: ER+_vs_others(455), HER2+_vs_others(100), TNBC_vs_others(254)
                   Plasma cell: ER+_vs_others(96), HER2+_vs_others(43), TNBC_vs_others(201)
                   T cell CD4+: ER+_vs_others(298), HER2+_vs_others(120), TNBC_vs_others(449)
                   T cell CD8+: ER+_vs_others(231), HER2+_vs_others(123), TNBC_vs_others(778)
Getting fold changes .. done. 


In [68]:
add_entrez <- function( df.deg.res, species )
{
    if(species == 'hsa'){
        org.db <- org.Hs.eg.db
    } else{
        org.db <- org.Mm.eg.db
    }

    my.symbols <- rownames(df.deg.res)
    suppressMessages( Convert_result <- biomaRt::select(org.db, keys = my.symbols,
                             columns = c("ENTREZID", "SYMBOL"),
                             keytype = "SYMBOL") )
    Convert_result <- distinct(Convert_result, SYMBOL, .keep_all= TRUE)

    df.deg.res <- cbind(df.deg.res, Convert_result)
    return(df.deg.res)
}

get_species <- function( species )
{
    if( tolower(species) %in% c('hs', 'human') ){ species = 'hsa' }
    else if( tolower(species) %in% c('mm', 'mouse')){ species = 'mmu' }
    return(species)
}

get_fold_changes <- function( df.deg.res, species, pval.cutoff = 0.01 )
{
    if( tolower(species) %in% c('hs', 'human') ){ species = 'hsa' }
    else if( tolower(species) %in% c('mm', 'mouse')){ species = 'mmu' }

    if( !("ENTREZID" %in% colnames(df.deg.res)) )
    {
        df.deg.res <- add_entrez( df.deg.res, species )
    }

    b <- (df.deg.res$pval <= pval.cutoff)
    foldchanges = df.deg.res$log2_FC[b]
    names(foldchanges) = df.deg.res$ENTREZID[b]

    return(foldchanges)
}


get_all_fold_changes <- function( lst.deg.res, species, pval.cutoff = 0.01 )
{
    cat(sprintf('Getting fold changes .. \n'))
    flush.console()

    lst.fc.res <- c()
    for( j in 1:length(lst.deg.res) )
    {
        cat(sprintf('%30s: ', names(lst.deg.res)[j]))
        flush.console()

        lst.deg.t <- lst.deg.res[[j]]
        lst.fc.t <- list()
        for( i in 1:length(lst.deg.t) )
        {
            foldchanges <- get_fold_changes(lst.deg.t[[i]], species, pval.cutoff )
            lst.fc.t[[i]] <- foldchanges
            if( i == length(lst.deg.t) )
            {
                cat(sprintf('%s(%d)\n', names(lst.deg.t)[i], sum(foldchanges != 0)))
            } else{
                cat(sprintf('%s(%d), ', names(lst.deg.t)[i], sum(foldchanges != 0)))
            }
        }
        names(lst.fc.t) <- names(lst.deg.t)
        lst.fc.res[[j]] <- lst.fc.t 
    }
    names(lst.fc.res) <- names(lst.deg.res)
    cat(sprintf('Getting fold changes .. done. \n'))
    return( lst.fc.res )
}


get_kegg_pathways_info <- function( species )
{
    kegg.db <- kegg.gsets(species = species, id.type = "kegg", check.new = TRUE)
    kegg_pw_id_name <- names(kegg.db$kg.sets)

    kegg_pw_id <- vapply(strsplit(kegg_pw_id_name, " ", fixed = TRUE), "[", "", 1)
    kegg_pw_name <- as.vector(vapply(kegg_pw_id_name, function(x) substring(x, 10), character(1)))

    df_pw <- data.frame( pw_id = kegg_pw_id, pw_name = kegg_pw_name, pw_id_name = kegg_pw_id_name )
    rownames(df_pw) <- (kegg_pw_id_name)

    pathways_kegg_entrez <- list()
    for( i in 1:length(kegg_pw_id_name) )
    {
        pathways_kegg_entrez[[i]] <- kegg.db$kg.sets[[kegg_pw_id_name[i]]]
    }
    names(pathways_kegg_entrez) <- kegg_pw_id_name

    return( list(id_name_map = df_pw, gene.sets = pathways_kegg_entrez ) )
}


convert_gene_symbols_in_Pathways_DB_to_entrez <- function(pathways_used, species)
{
    if( tolower(species) %in% c('hs', 'human') ){ species = 'hsa' }
    else if( tolower(species) %in% c('mm', 'mouse')){ species = 'mmu' }

    if(species == 'hsa'){
        org.db <- org.Hs.eg.db
    } else{
        org.db <- org.Mm.eg.db
    }

    pathways_names_used <- names(pathways_used)
    ## For each pathway, convert hugo symbols into ENTREZ symbol
    pathways_used_entrez <- list()
    cat(sprintf('Converting Pathways DB .. \r'))
    for( i in 1:length(pathways_names_used) )
    {
        cat(sprintf('Converting Pathways DB .. %3d/%3d   \r', i, length(pathways_names_used)) ) #, pathways_names_used[i]))
        flush.console()

        hugo.symbols <- pathways_used[[pathways_names_used[i]]]
        if( length(hugo.symbols) == 1 ){ hugo.symbols <- hugo.symbols[[1]] }

        {
            s <- hugo.symbols[[1]][length(hugo.symbols[[1]])]
            if(  substr( s, str_length(s), str_length(s) ) == '\n' )
            {
                hugo.symbols[[1]][length(hugo.symbols[[1]])] <- substr( s, 1, str_length(s)-1 )
            }
        }

        suppressMessages( Convert_result <- biomaRt::select(org.db, keys = hugo.symbols,
                                 columns = c("ENTREZID", "SYMBOL"),
                                 keytype = "SYMBOL") )
        Convert_result <- distinct(Convert_result, SYMBOL, .keep_all= TRUE)
        pathways_used_entrez[[i]] <- Convert_result[,"ENTREZID"]
        wh <- which(is.na( pathways_used_entrez[[i]] ))
        pathways_used_entrez[[i]] <- pathways_used_entrez[[i]][-wh]
    }
    names(pathways_used_entrez) <- pathways_names_used
    cat(sprintf('Converting Pathways DB .. done.        \n'))

    return(pathways_used_entrez)
}


## For each pathway, find the best-match KEGG pathways
build_pathway_map <- function(pathways_used_entrez, pathways_kegg_entrez, o_th = 0.85)
{
    # o_th <- 0.85
    pathways_map <- c()
    names_map <- c()

    cnt <- 0
    for( i in 1:length(pathways_used_entrez) )
    {
        ov <- c()
        or <- c()
        for( j in 1:length(pathways_kegg_entrez) )
        {
            pwi <- intersect( pathways_used_entrez[[i]], pathways_kegg_entrez[[j]] )
            ov[j] <- length(pwi)
            or[j] <- length(pwi) /min(length(pathways_used_entrez[[i]]), length(pathways_kegg_entrez[[j]]))
        }
        w <- which.max(ov)
        if(or[w] >= o_th)
        {
            cnt <- cnt + 1
            pathways_map[cnt] <- names(pathways_kegg_entrez)[w]
            names_map[cnt] <- names(pathways_used_entrez)[i]
        }
    }
    names(pathways_map) <- names_map

    return(pathways_map)
}


select_valid_pathways_from_gsa_result <- function( df.gsa, pathways_map, df_kegg_pw_map,
                                                   p.val.cutoff = 0.01, verbose = FALSE )
{
    p.val.cutoff.scoda <- p.val.cutoff
    pv.col <- 'pval'

    n_before <- dim(df.gsa)[1]
    b <- df.gsa[,pv.col] <= p.val.cutoff.scoda
    df.gsa <- df.gsa[b,]
    pw_detected_scoda <- ( pathways_map[rownames(df.gsa)] )
    n_after <- dim(df.gsa)[1]
    pw_detected_scoda <- unique(pw_detected_scoda)

    if( verbose )
    {
        cat(sprintf('# GSA items: %d -> %d -> %d -> ', n_before, n_after, length(pw_detected_scoda) ))
    }

    kegg_pw_id_name <- rownames(df_kegg_pw_map)
    pw_scoda <- sort(pw_detected_scoda)
    pw_kegg <- sort((kegg_pw_id_name))
    pw_common <- intersect(pw_scoda, pw_kegg)
    pw_scoda_only <- setdiff(pw_scoda, pw_common)
    pw_kegg_only <- setdiff(pw_kegg, pw_common)

    n_common <- length(pw_common)
    n_scoda <- length(pw_scoda)
    n_scoda_only <- length(pw_scoda_only)
    n_kegg <- length(pw_kegg)

    if( verbose )
    {
        cat(sprintf('%d intersection %d -> %d \n', n_scoda, n_kegg, n_common ))
    }
    df_pw_sel <- df_kegg_pw_map[pw_common, ]

    return(df_pw_sel)
}


get_pathways_map <- function(pathways_used, species, min_overlap = 0.85 )
{
    if( tolower(species) %in% c('hs', 'human') ){ species = 'hsa' }
    else if( tolower(species) %in% c('mm', 'mouse')){ species = 'mmu' }

    pathways_used_entrez <- convert_gene_symbols_in_Pathways_DB_to_entrez( pathways_used, species )

    lst <- get_kegg_pathways_info( species = species )
    df_kegg_pw_map <- lst$id_name_map
    pathways_kegg_entrez <- lst$gene.sets

    pathways_map <- build_pathway_map( pathways_used_entrez, pathways_kegg_entrez, o_th = min_overlap )

    pws <- names(pathways_map)

    pw_sel <- c()
    cnt <- 0
    for( pw in pws )
    {
        if( pathways_map[[pw]] %in% rownames(df_kegg_pw_map) )
        {
            cnt <- cnt + 1
            pw_sel[cnt] <- pw
        }
    }

    df_kegg_pw_map_sel <- df_kegg_pw_map[ pathways_map[pw_sel], ]
    df_kegg_pw_map_sel[,'pw_name_used'] <- pw_sel

    return(df_kegg_pw_map_sel)
    # return( list(kegg_pw_map = df_kegg_pw_map, pathways_map = pathways_map) )
}


save_kegg_pathviews <- function( target_cell, lst.gsa.all, lst.fcs.all, df_pathways_map, 
                                 species, gsa.p.val.cutoff = 0.01 )
{
    if( tolower(species) %in% c('hs', 'human') ){ species = 'hsa' }
    else if( tolower(species) %in% c('mm', 'mouse')){ species = 'mmu' }

    lst.df.gsa <- lst.gsa.all[[target_cell]]
    fcs.entrez <- lst.fcs.all[[target_cell]]

    pathways_map <- rownames(df_pathways_map)
    names(pathways_map) <- df_pathways_map[,'pw_name_used']

    df_kegg_pw_map <- df_pathways_map

    dir_to_save <- sprintf('KEGG_pathview_%s', target_cell )

    if( !file.exists(dir_to_save) ){
        dir.create(dir_to_save)
    } else{
        f.lst <- list.files(dir_to_save)
        for(i in 1:length(f.lst))
        {
            f <- paste0(dir_to_save, '/', f.lst[i])
            if( file.exists(f) ){ file.remove(f) }
        }
    }
    items <- names(lst.df.gsa)

    for( j in 1:length(items) )
    {
        item <- items[j]
        foldchanges <- fcs.entrez[[item]]

        df.gsa <- lst.df.gsa[[item]]
        if( dim(df.gsa)[1] > 0 )
        {
            df_pw_sel <- select_valid_pathways_from_gsa_result( df.gsa, pathways_map, df_kegg_pw_map,
                                                                p.val.cutoff = gsa.p.val.cutoff )

            if( dim(df_pw_sel)[1] > 0)
            {
                pw_id_sel <- df_pw_sel$pw_id
                pw_name_sel <- df_pw_sel$pw_name

                if( length(pw_name_sel) == 1 )
                {
                    s_max = str_length(pw_name_sel)
                } else
                {
                    s_max = 0
                    for( i in 1:length(pw_name_sel)){ 
                        s_max <- max( s_max, str_length(pw_name_sel[i]) ) 
                    }
                }    
                for( i in 1:length(pw_name_sel))
                {
                    pid <- pw_id_sel[i]
                    pname <- pw_name_sel[i]
                    pname <- str_replace( pname, '/', '_' )
    
                    s_suffix = ''
                    if( str_length(pname) < s_max )
                    {
                        for( k in 1:(s_max - str_length(pname)) )
                        {
                            s_suffix <- paste0(s_suffix, ' ')
                        }
                    }
    
                    cat(sprintf('%30s: %d/%d - %d/%d - %s%s \r', target_cell, j, length(items),
                                  i, length(pw_name_sel), pname, s_suffix ))
                    flush.console()
    
                    suppressMessages( pv.out <- pathview(gene.data = foldchanges, pathway.id=pid,
                              species=species, kegg.dir = dir_to_save, # out.suffix = 'pos',
                              low = list(gene = "turquoise", cpd = "blue"),
                              mid = list(gene = "gray", cpd = "gray"),
                              high = list(gene = "gold", cpd = "yellow"),
                              kegg.native = TRUE, same.layer = FALSE, min.nnodes = 5))
    
                    if( is.list(pv.out) )
                    {
                        file_out <- paste0(pname, '_', item, '.png')
                        file.rename(paste0(pid,'.pathview.png'), file_out)
                        suppressMessages( file.move(file_out, dir_to_save) )
                        if( file.exists(paste0(dir_to_save, '/', pid,'.png')) )
                        {
                            file.remove(paste0(dir_to_save, '/', pid,'.png'))
                        }
                        if( file.exists(paste0(dir_to_save, '/', pid,'.xml')) )
                        {
                            file.remove(paste0(dir_to_save, '/', pid,'.xml'))
                        }
                    }
                }
            }
        }
    }
    cat(sprintf('%30s: %d/%d - %d/%d - %s%s \n', target_cell, j, length(items),
                  i, length(pw_name_sel), pname, s_suffix ))
    return(dir_to_save)
}


In [69]:
target_cell <- 'Aneuploid unassigned'
dir_saved <- save_kegg_pathviews( target_cell, lst.gsa.all,
                                  lst.fcs.all, df_pathways_map,
                                  species, gsa.p.val.cutoff = 1e-4 )

          Aneuploid unassigned: 3/3 - 5/5 - Parkinson's disease                         


In [73]:
dir_saved

[1] "KEGG_pathview_Epithelial cell"

In [ ]:
names(lst.fcs.all)

[1] "Aneuploid Epithelial cell" "B cell"                   
 [3] "Diploid Epithelial cell"   "Endothelial cell"         
 [5] "Epithelial cell"           "Fibroblast"               
 [7] "ILC"                       "Macrophage"               
 [9] "Plasma cell"               "Smooth muscle cell"       
[11] "T cell CD4+"               "T cell CD8+"

In [ ]:
for( target_cell in names(lst.fcs.all) )
{
    dir_saved <- save_kegg_pathviews( target_cell, lst.gsa.all, lst.fcs.all, df_pathways_map, species, 
                                      gsa.p.val.cutoff = 0.01 )
}